# Blockchain based ROSCA - version 2
---

* [System Requirements (Part 1)](#System-Requirements)
  * [Model Introduction](#Model-Introduction)
  * [Requirements Analysis](#Requirements-Analysis)
  * [Differential Game](#Differential-Game)
* [System Design (Part 2)](#System-Design)
  * [Differential Specification](#Differential-Specification)
    0. [Dependencies](#0.-Dependencies)
    1. [State Variables](#1.-State-Variables)
    2. [Reward + Cost](#2.-Reward-+-Cost)
    3. [Agent Strategies Definition](#2.-Agent-Strategies-Definition)
    4. [Agent Behaviours Definition](#2.-Agent-Behaviours-Definition)
    5. [Policy Functions](#3.-Policy-Functions)
    6. [State Update Functions](#4.-State-Update-Functions)
    7. [Partial State Update Blocks](#5.-Partial-State-Update-Blocks)
    8. [Configuration](#6.-Configuration)
    9. [Execution](#7.-Execution)
    10. [Simulation Output Preparation](#8.-Simulation-Output-Preparation)
    11. [Simulation Analysis](#9.-Simulation-Analysis)  
---

# System Requirements

## Model Introduction
In this notebook we improve on our first ROSCA model, a blockchain based simulation environment for informal savings and credit institutions. 

Previously, we layed out the basic dynamics of a blockchain based rosca and implemented the core mechanisms: the ability to contribute and recieve money from the rosca circle. We also identified our initial models limitations: namely, the lack of dimension to the agents and incentives to influence decisions. 

Since we are modelling a social and economic system, we are constantly asking what our agents will do. Answering this is a second step towards an abstracted (but realistic) simulation environment, where we essentially outline the economic incentives associated with the choices the agents can make. More specifically, what economics incentives are associated with the choice to behave honestly, or dishonetly. 

In the worse case senario, multiple agents from the same circle behave dishonestly, resulting in the rest of the agents not recieving their full amount they contributed (as seen in version 1) or disincentiving them to continue contributing. How can we create policies that are robust to network uncertainty and strategic interactions?

We will develop these questions throughout, getting insight from our models and simulations.

## Requirements Analysis

For this version, we extend our ROSCA model to focus on two things: 
1. Producing rich agent-based models 
2. Programming incentives and seeing how agent behaviours map to these the incentives (rewards, penalties, general outcomes). 


## Differential Game

> Game theory is the study of the ways in which interacting choices of economic agents produce outcomes with respect to the preferences (or utilities) of those agents

> In game theory, differential games are a group of problems related to the modeling and analysis of conflict in the context of a dynamical system

### Agents' Dilemma

In this simulation the agents are faced with two choices: 
1. To cheat and steal the pot - an agent is faced with this choice once for the entire duration 
2. To contribute money - an agent is faced with this choice every timestep (month)

We explore these choices using a differential game. Assuming the choice to steal the pot is expensive (incurs a heavy cost) and agents do not undertake this task unless their is an expected ROI is greater that penalties incurred or the incentives associated good behaviour such as: a higher credit-score. 

![Feedback](./images/feedback.png)

At each point in time both Contributer's and Cheaters must make a decision about how to act. 
Cheaters and Contributers beliefs about the expected returns on their actions as estimated by their observations of the other's actions over time.
For an example: if an agent sees others either aren't contributing or have stolen the pool, this will heavily influence their decision on wether to continue behaving honestly.

![Flow](./images/flow_v2.png)

---

# System Design

## Differential Specification

![Differential Specification](./images/diff.png)

In this notebook, we explore the "Non-Equibrium Dynamics" that arise from introducing ROI estimation as feedback to the 'User decision' policy of users. 

## 0. Dependencies

In [34]:
# Standard libraries: https://docs.python.org/3/library/
import math
import secrets
import random
from datetime import timedelta

# cadCAD configuration modules
from cadCAD import configs
from cadCAD.configuration import Configuration
from cadCAD.configuration.utils import exo_update_per_ts, bound_norm_random, ep_time_step, config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

In [35]:
# Logic
from circle import initialise_circle_state
from chain import create_block
from agents import (AgentData, update_contribution_amount, verify_allocated_amount, get_monthly_default_count, calculate_agent_amount)

## 1. State Variables

### Genesis state

In [36]:
#used with stochastic processes to make experiments repeatable
seed = {}

# define agents each agreeing to contribute 300£ a month
def get_initial_deposits(n):
    agent = [AgentData(secrets.token_bytes(48), 300, i, 0, False, 0, False, 0)
             for i in range(n)]       
    return agent

agent_genesis = get_initial_deposits(10)

In [37]:
# Initial Conditions
genesis_state = {
    'Contributers_On': True, # contributers are active
    'Cheaters_On': False, # cheaters are inactive
    'Total_Volume': 0, # total volume of pool activity (GBP)
    'Honest_Volume': 0, # volume of honest (contributed) activity (GBP)
    'Dishonest_Volume': 0, # volume of dishonest (non-contributed) activity (GBP)
    'Cheats_Volume': 0, # volume of cheater activity (GBP)
    'Contributers_Rating': 0, # ratings of contributing agent
    'Contributers_Cost': 0, # cost incurred by contributers
    'Contributers_Reward': 0, # rewards collected by verifiers
    'Cheaters_Cost': 0, # costs incurred by cheaters
    'Cheater_Reward': 0, # rewards (profit) achieved by cheating 
    'timestep': 0,
    'agent_data': agent_genesis,
    'Duration': 10
}

# 2. Reward + Cost

> Here we outline our State Dependant Reward and Cost Functions which characterise the players utilities 

In [38]:
# contributers's reward per round contributed => amount agreed
def contributers_reward(data, s):
    return calculate_agent_amount(data, s['timestep'], s['Duration'])

# Contributer's cost per round cheater acts => amount defaulted if at all
def contributer_cost(s):
    share = 0.1
    return (share * s['Cheats_Volume'])

# Cheater's reward per cheat
def cheater_reward(s, allocation_amt):
    print("check allocation_amt: " + str(allocation_amt) + " contributed: " + str(s.contributed_amt))
    return allocation_amt - s.contributed_amt # Pot amount - Contributed amount

# Cheater's cost per cheat caught
delta = 0.05
def cheater_cost(s, allocation_amt):
    return (delta * allocation_amt) + allocation_amt - s.contributed_amt

# 3. Agent Strategies Definition
> Here we define our behavioral models in which agents interact according to their own heuristic strategies

#### Three types of contributers:

In [39]:
def honest_contributer(reward, cost):
    if(cost > reward): # contributes just as long as cost isn't greater than reward 
       return False
    else :
        return True    


def influenced_contributer(reward, cost):
    if((0.25 * cost) > reward): # gets discouraged from contributing if cost > 25% of reward
       return False
    else :
        return True 

def broke_contributer(reward, cost):       
    if(random.random() < 0.1): # 10% chance this contributer doesn't have enough money to contribute
       return False
    else :
        return True 



#### Two types of 'cheaters' (those who are allocated the pot once each timestep):

In [40]:
def honest_allocator(reward, cost):   
    if(cost > reward): # doesn't steal pot as long as cost isnt greater than reward 
       return False
    else:
        return True    

def greedy_allocator(reward, cost):   
    risk = random.random()
    if((reward > cost) and risk < 0.1): # if reward is greater than cost and agent accepts risk of not being caught
        return True
    else:
        return False  

# 4. Agent Behaviours Definition
> We can now define our two types of agents

#### 1. A contributer 
Contributer motivated to contribute their share if reward is greater than the cost of other agents stealing 

In [41]:
def p_contributer(state, agent):
    act = False
    cost = contributer_cost(state)
    reward = contributers_reward(agent, state)

    contributer_strategies = [honest_contributer, influenced_contributer, broke_contributer]
    choose_strategy = random.choice(contributer_strategies)
    
    if(agent.defaulted):
        return act

    print("Shall I contribute? reward: " + str(reward) + " cost: " + str(0.25 * cost) + " strategy: " + str(choose_strategy))
    if (choose_strategy(reward, cost)):
        act = True
        agent.rating += 0.2 # increase rating 
        state['Contributers_Rating'] += 0.2
        agent.contributed_amt += agent.amount # update agent contributed amount

    return act

#### 2. A cheater
Cheater motivated to steal the pot if reward is greater than their amount contributed so far + can take the risk of being caught

In [42]:
def p_cheater(state, agent, allocation_amt):
    act = False
    risk = random.random()
    reward = cheater_reward(agent, allocation_amt)
    cost = cheater_cost(agent, allocation_amt)

    strategies = [honest_allocator, greedy_allocator]
    choose_strategy = random.choice(strategies)
    print("Shall I steal? reward: " + str(reward) + " cost: " + str(cost) + " strategy: " + str(choose_strategy))

    if(choose_strategy(reward, cost)):
        print("YESSSS - YOLO")
        act = True
        state['Cheats_Volume'] += reward 
        state['Cheater_Reward'] = reward
        state['Cheaters_Cost'] = cost

    return act

# 5. Policy Functions

> In this section Policies are defined for our cheaters and contributers choices.

#### In our differential specification, we focus on two policies:  
1. A system decision - Monthly collection policy which leads to -> A players decision - the decision to contribute the amount set out or not

We can now define these policies:

In [43]:
def p_intitiate_monthly_deposits(params, substep, state_history, previous_state):
    month_sum = 0.0
    expected_month_sum = 0.0
    agent_data = previous_state['agent_data']
    month = previous_state['timestep']

    print("\n" + "STARTING ROSCA MONTH " + str(month))
    
    # for each agent add their monthly contribution to the collected pool
    for x in range(0, len(agent_data)):
        agent_contribution = agent_data[x].amount
        expected_month_sum += agent_contribution

        # player decision - initiate contributers behaviour
        if(p_contributer(previous_state, agent_data[x]) == True):
            month_sum += agent_contribution # c - from differential eq
        else :
            print("Contributer default: GBP" + str(agent_contribution))

    previous_state['Total_Volume'] = expected_month_sum
    previous_state['Honest_Volume'] = month_sum
    previous_state['Dishonest_Volume'] = expected_month_sum - month_sum

    print("Total deposited amount: GBP" + str(previous_state['Honest_Volume']) + " - Defaulted amount: " + str(previous_state['Dishonest_Volume']))

    return {'collected_amount': month_sum}



2. A system decision - Monthly allocation policy which leads to -> A players decision - the decision to steal the pool allocated to you or not

In [44]:
def p_allocate_pool(params, substep, state_history, previous_state):
    block_agent_data = previous_state['agent_data']
    pool_amount = previous_state['Honest_Volume']
    timestep = previous_state['timestep']

    for x in range(0, len(block_agent_data)):
        if(block_agent_data[x].standing == timestep and block_agent_data[x].taken == False):
            allocation_amt = calculate_agent_amount(block_agent_data[x], timestep+1, previous_state['Duration']) # a - from differential eq

            print("verify: allocation_amt: " + str(allocation_amt) + ", pool_amount" + str(pool_amount))
            if(verify_allocated_amount(block_agent_data[x], allocation_amt, pool_amount) == True):    
                block_agent_data[x].taken = True
                pool_amount -= allocation_amt

                print("Allocating agent: " + str(block_agent_data[x].pubkey) + ", standing: " + str(block_agent_data[x].standing) + ", amount: GBP" + str(allocation_amt))
                
            else :
                print("Not enough money in pool to allocate agent full amount of: " + str(allocation_amt) + "\n")    
                allocation_amt = pool_amount
                block_agent_data[x].taken = False
                pool_amount = 0.0

            # player decision - initiate cheater behaviour
            if(p_cheater(previous_state, block_agent_data[x], allocation_amt) == True):
                previous_state['Contributers_Cost'] = contributer_cost(previous_state)
                previous_state['Cheaters_On'] = True
                print("Allocator default: GBP" + str(previous_state['Cheater_Reward']))
                block_agent_data[x].defaulted = True

            block_agent_data[x].balance += allocation_amt
    return {'pool_amount' : pool_amount}

# 6. State Update Functions

So far we have described the system of rewards and the simple strategies being evaluated, but in order to implement a differential game one must also explicit encode the mechanisms. Mechanisms are the functions that map decisions to changes in the system state.

In [45]:
def s_updatepool(params, substep, state_history, previous_state, policy_input):
    """
    Update the pool state according to the differential equation (1):
    current_pool_state + collected_amount - allocation_amount
    """
#     pool = previous_state['pool_state']
#     duration = params['duration']
    
    p_collected_amt = policy_input['collected_amount']
    p_pool_amt = policy_input['collected_amount']
    
    return 'pool_state', genesis_state

# 7. Partial State Update Blocks

In [46]:
partial_state_update_blocks = [
    {
        'policies': {
            'collected_amount': p_intitiate_monthly_deposits,
            'pool_amount': p_allocate_pool
        }, 
        # State variables
        'variables': {
            'pool_state': s_updatepool            
        }
    }
]

# 8. Configuration

In [47]:
system_params = {
    'duration': [10], # duration of rosca circle
}
system_params

sim_config = config_sim({
    "N": 1,
    "T": range(10),
#     "M": system_params
})

from cadCAD import configs
del configs[:] # Clear any prior configs

experiment = Experiment()
experiment.append_configs(
    initial_state = genesis_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)
configs[-1].__dict__

{'sim_config': {'N': 1,
  'T': range(0, 10),
  'M': [{}],
  'subset_id': 0,
  'subset_window': deque([0, None]),
  'simulation_id': 0,
  'run_id': 0},
 'initial_state': {'Contributers_On': True,
  'Cheaters_On': False,
  'Total_Volume': 0,
  'Honest_Volume': 0,
  'Dishonest_Volume': 0,
  'Cheats_Volume': 0,
  'Contributers_Rating': 0,
  'Contributers_Cost': 0,
  'Contributers_Reward': 0,
  'Cheaters_Cost': 0,
  'Cheater_Reward': 0,
  'timestep': 0,
  'agent_data': [<agents.AgentData at 0x11fd718e0>,
  'Duration': 10},
 'seeds': {},
 'env_processes': {},
 'exogenous_states': {},
 'partial_state_updates': [{'policies': {'collected_amount': <function __main__.p_intitiate_monthly_deposits(params, substep, state_history, previous_state)>,
    'pool_amount': <function __main__.p_allocate_pool(params, substep, state_history, previous_state)>},
   'variables': {'pool_state': <function __main__.s_updatepool(params, substep, state_history, previous_state, policy_input)>}}],
 'policy_ops': [<func

In [48]:
# configs.append(
#     Configuration(
#         sim_config=config_sim,
#         state_dict=genesis_states,
#         seed=seed,
# #         exogenous_states=exogenous_states,
#         env_processes=env_processes,
#         mechanisms=mechanisms
#     )
# )

# 7. Execution

In [49]:
from cadCAD.engine import ExecutionMode, ExecutionContext, Executor
# from demos import simple_tracker_config
from cadCAD import configs
exec_context = ExecutionContext()
simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (10, 1, 1, 14)
Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [0]
Execution Mode: single_threaded

STARTING ROSCA MONTH 0
calculate_agent_amount: 0 0
Shall I contribute? reward: 3000 cost: 0.0 strategy: <function influenced_contributer at 0x11fc63670>
calculate_agent_amount: 0 0
Shall I contribute? reward: 3000 cost: 0.0 strategy: <function influenced_contributer at 0x11fc63670>
calculate_agent_amount: 0 0
Shall I contribute? reward: 3000 cost: 0.0 strategy: <function influenced_contributer at 0x11fc63670>
calculate_agent_amount: 0 0
Shall I contribute? reward: 3000 cost: 0.0 strategy: <function broke_contributer at 0x11fc63

# 8. Simulation Output Preparation

In [50]:
import pandas as pd
import numpy as np

result = pd.DataFrame(raw_result)
result.head(20)

,Contributers_On,Cheaters_On,Total_Volume,Honest_Volume,Dishonest_Volume,Cheats_Volume,Contributers_Rating,Contributers_Cost,Contributers_Reward,Cheaters_Cost,Cheater_Reward,timestep,agent_data,Duration,simulation,subset,run,substep,pool_state
0,True,False,0.0,0.0,0.0,0,0.0,0,0,0,0,0,[<class 'agents.AgentData'>: {'pubkey': b'\xdf...,10,0,0,1,0,NaN
1,True,False,3000.0,3000.0,0.0,0,2.0,0,0,0,0,1,[<class 'agents.AgentData'>: {'pubkey': b'\xdf...,10,0,0,1,1,"{'Contributers_On': True, 'Cheaters_On': False..."
2,True,False,3000.0,2700.0,300.0,0,3.8,0,0,0,0,2,[<class 'agents.AgentData'>: {'pubkey': b'\xdf...,10,0,0,1,1,"{'Contributers_On': True, 'Cheaters_On': False..."
3,True,False,3000.0,3000.0,0.0,0,5.8,0,0,0,0,3,[<class 'agents.AgentData'>: {'pubkey': b'\xdf...,10,0,0,1,1,"{'Contributers_On': True, 'Cheaters_On': False..."
4,True,False,3000.0,3000.0,0.0,0,7.8,0,0,0,0,4,[<class 'agents.AgentData'>: {'pubkey': b'\xdf...,10,0,0,1,1,"{'Contributers_On': True, 'Cheaters_On': False..."
5,True,False,3000.0,2700.0,300.0,0,9.6,0,0,0,0,5,[<class 'agents.AgentData'>: {'pubkey': b'\xdf...,10,0,0,1,1,"{'Contributers_On': True, 'Cheaters_On': False..."
6,True,False,3000.0,3000.0,0.0,0,11.6,0,0,0,0,6,[<class 'agents.AgentData'>: {'pubkey': b'\xdf...,10,0,0,1,1,"{'Contributers_On': True, 'Cheaters_On': False..."
7,True,False,3000.0,2700.0,300.0,0,13.4,0,0,0,0,7,[<class 'agents.AgentData'>: {'pubkey': b'\xdf...,10,0,0,1,1,"{'Contributers_On': True, 'Cheaters_On': False..."
8,True,False,3000.0,3000.0,0.0,0,15.4,0,0,0,0,8,[<class 'agents.AgentData'>: {'pubkey': b'\xdf...,10,0,0,1,1,"{'Contributers_On': True, 'Cheaters_On': False..."
9,True,False,3000.0,2400.0,600.0,0,17.0,0,0,0,0,9,[<class 'agents.AgentData'>: {'pubkey': b'\xdf...,10,0,0,1,1,"{'Contributers_On': True, 'Cheaters_On': False..."


In [ ]:
## store

# # these mechanisms are for state updates are computed from other states
# # due to the simplicity of this model; 
# # exogenous states, and exogenous process features of the simulation framework are used
# # these models can also be handled as standard states and mechanism 
# # with only minor changes to the config

# #the total volume is modeled as static but changing epsilon to a random variable
# #is a simple way to introduce a stochastic process to make Monte Carlo simulations interesting
# epsilon = 1
# def volume_ep(params, substep, state_history, previous_state, policy_input):
#     y = 'Total_Collected'
#     x = epsilon*previous_state['Total_Collected']
#     return (y, x)

# #zeta is the percentage of the total volume which is cheating volume
# #extend this model by making zeta an output of a Policy or a random variable
# zeta=0.2
# def cheat_volume_ep(params, substep, state_history, previous_state, policy_input):
#     y = 'Cheats_Volume'
#     if (previous_state['Cheaters_On']):
#         x = zeta*(previous_state['Total_Collected'])
#     else:
#         x = 0
#     return (y, x)

# #honest volume is the complement of the cheat volume
# def honest_volume_ep(params, substep, state_history, previous_state, policy_input):
#     y = 'Honest_Volume'
#     if (state_history['Cheaters_On']):
#         x = (1-zeta)*previous_state['Total_Collected']
#     else:
#         x = previous_state['Total_Collected']
#     return (y, x)

# #resolve the cheat volume caught being caught
# # def cheats_caught_ep(step, sL, s, _input):
# #     y = 'Cheats_Caught_Volume'
# #     if (s['Contributers_On']):
# #         x = s['Cheats_Volume']
# #     else:
# #         x = 0
# #     return (y, x)

# #resolve the costs incurred by verifiers verifying transactions
# def verifier_cost_ep(params, substep, state_history, previous_state, policy_input):
#     y = 'Contributers_Cost'
#     if (previous_state['Contributers_On']):
#         x = verifier_cost(previous_state)
#     else:
#         x = 0
#     return (y, x)

# #resolve the rewards earned by verifiers verifying transactions
# def verifier_reward_ep(params, substep, state_history, previous_state, policy_input):
#     y = 'Contributers_Reward'
#     if (previous_state['Contributers_On']):
#         x = verifier_reward(previous_state)
#     else:
#         x = 0
#     return (y, x)

# #resolve the costs incurred by cheaters getting caught
# def cheater_cost_ep(params, substep, state_history, previous_state, policy_input):
#     y = 'Cheaters_Cost'
#     if (previous_state['Contributers_On']):
#         x = cheater_cost(previous_state)
#     else:
#         x = 0
#     return (y, x)

# #resolve the rewards won by cheaters getting away with cheating
# def cheater_reward_ep(params, substep, state_history, previous_state, policy_input):
#     y = 'Cheater_Reward'
#     if (previous_state['Cheaters_On']):
#         x = cheater_reward(previous_state)
#     else:
#         x = 0
#     return (y, x)


# # store the state and state update functions 
# #as "exogenous_states" updated as "ep" = "exogenous process"
# # note this terminology is being deprecated 
# # as it confuses internal dynamics with models of processes
# # which occur outside of but have influence on our system
# # exogenous_states = exo_update_per_ts(
# #     {
# #     'Total_Collected': volume_ep(substep, state_history, previous_state, policy_input),
# #     'Honest_Volume': honest_volume_ep,
# #     'Cheats_Volume': cheat_volume_ep,
# # #     'Cheats_Caught_Volume': cheats_caught_ep,
# #     'Contributers_Cost': verifier_cost_ep,
# #     'Contributers_Reward': verifier_reward_ep,
# #     'Cheaters_Cost': cheater_cost_ep,
# #     'Cheater_Reward': cheater_reward_ep,
# #     'timestamp': time_model
# #     }
# # )

# #not used in this simulation
# env_processes = {
# }